In [1]:
import pandas as pd
from pandas import json_normalize
import ast

In [2]:
data_list = []
#Ruta del archivo JSON
file_path = "datasets/australian_users_items.json"

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
items = pd.DataFrame(data_list)

In [3]:
data_it1 = items.explode(['items'])
data_it2 = pd.json_normalize(data_it1['items']).set_index(data_it1['items'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

In [4]:
data_it3.drop(['user_url','steam_id'], axis=1, inplace=True)

In [5]:
data_it3 = data_it3.dropna(subset=['item_id'])
data_it3 = data_it3.dropna(subset=['playtime_forever'])

In [6]:
data_it3.drop(['items','playtime_2weeks'], axis=1, inplace=True)

In [7]:
data_it3 = data_it3[data_it3['playtime_forever'] != 0]

In [8]:
data_it3 = data_it3[['user_id','item_id','playtime_forever', 'items_count', 'item_name']]

In [9]:
data_it3

,user_id,item_id,playtime_forever,items_count,item_name
0,76561197970982479,10,6.0,277,Counter-Strike
0,76561197970982479,30,7.0,277,Day of Defeat
0,76561197970982479,300,4733.0,277,Day of Defeat: Source
0,76561197970982479,240,1853.0,277,Counter-Strike: Source
0,76561197970982479,3830,333.0,277,Psychonauts
...,...,...,...,...,...
88308,76561198329548331,304930,677.0,7,Unturned
88308,76561198329548331,227940,43.0,7,Heroes & Generals
88308,76561198329548331,388490,3.0,7,One Way To Die: Steam Edition
88308,76561198329548331,521570,4.0,7,You Have 10 Seconds 2


### Verificamos si es necesario que la columna playtime_forever sea float, checkeando si alguna fila tiene algun valor decimal

In [10]:
parte_decimal = data_it3['playtime_forever'].apply(lambda x: x - int(x))

# Verificar si todos los valores de la parte decimal son cero
todos_ceros = parte_decimal.eq(0).all()

if todos_ceros:
    print("Todos los valores después de la coma son cero.")
else:
    print("Al menos un valor después de la coma no es cero.")

Todos los valores después de la coma son cero.


In [11]:
# La columna playtime_forever se refiere a minutos. Vamos a cambiarlo a horas, para que sea mas útil manejarlo a futuro y lo redondeamos a dos decimales
data_it3['playtime_forever'] = round(data_it3['playtime_forever']/60,2)

# Convertir la columna 'id' de float a int en el DataFrame original
data_it3['item_id'] = data_it3['item_id'].astype(int)

In [12]:
data_it3

,user_id,item_id,playtime_forever,items_count,item_name
0,76561197970982479,10,0.10,277,Counter-Strike
0,76561197970982479,30,0.12,277,Day of Defeat
0,76561197970982479,300,78.88,277,Day of Defeat: Source
0,76561197970982479,240,30.88,277,Counter-Strike: Source
0,76561197970982479,3830,5.55,277,Psychonauts
...,...,...,...,...,...
88308,76561198329548331,304930,11.28,7,Unturned
88308,76561198329548331,227940,0.72,7,Heroes & Generals
88308,76561198329548331,388490,0.05,7,One Way To Die: Steam Edition
88308,76561198329548331,521570,0.07,7,You Have 10 Seconds 2


In [13]:
data_it3.to_parquet("datasets/Parquet/items.parquet", index=False)